In [277]:
import csv
from bs4 import BeautifulSoup
from urllib2 import urlopen

teams = ['BOS','TOR','NYK','PHI','BRK','WAS','ATL','MIA','CHO','ORL','CLE','MIL','IND','CHI','DET','UTA','OKC','POR','DEN','MIN','SAS','HOU','MEM','NOP','DAL','GSW','LAC','SAC','LAL','PHO']

# Mode is set to 'a' for append. 'w' will overwrite
file = open('data.csv','wb')
mywriter = csv.writer(file)

def make_soup(url):
    html = urlopen(url).read()
    return BeautifulSoup(html, "lxml")

def get_player_names():
    file = open('player_names.txt','wb')
    file2 = open('player_ids.txt','wb')
    
    team_url = 'http://www.basketball-reference.com/teams/{team}/2017.html'
    for team in teams:
        url = team_url.format(team=team)
        soup = make_soup(url)
        tbody = soup.find('tbody')
        players = tbody.select('a[href*="/players/"]')
        for player in players:
            file.write(player.text.encode('utf-8') + '\n')
            file2.write(player.encode('utf-8')[18:29] + '\n')
            
    file.close()
    file2.close()

def get_player_positions():
    file = open('player_positions.txt', 'wb')
    team_url = 'http://www.basketball-reference.com/teams/{team}/2017.html'
    count = 1
    for team in teams:
        url = team_url.format(team=team)
        soup = make_soup(url)
        tbody = soup.find('tbody')
        positions = tbody.select('.center')
        for position in positions:
            if count % 2 == 0:
                file.write(position.text.encode('utf-8') + '\n')
            count += 1
file.close()

def get_team_stats(possession):
file = open('team_defensive_stats.csv', 'wb')
mywriter = csv.writer(file)

url = advanced_url.format(possession=possession)
soup = make_soup(url)
tbody = soup.find('tbody')

for tr in tbody.findAll('tr'):
    team_stats = []
    for td in tr.findAll('td'):
        team_stats.append(td.text.lstrip().encode('utf-8'))
    mywriter.writerow(team_stats)
file.close()

def get_stats(name, player, position):
    
    player_url = 'https://www.basketball-reference.com/players/{player}/gamelog/2017/'.format(player=player)
    player_advanced_url = 'https://www.basketball-reference.com/players/{player}/gamelog-advanced/2017/'.format(player=player)
    player_stats = []
   
    # Basic Stats
    url = player_url
    soup = make_soup(url)
    tbody = soup.find('tbody')

    # Advanced Stats
    url_advanced = player_advanced_url
    soup_advanced = make_soup(url_advanced)
    tbody_advanced = soup_advanced.find('tbody')

    for tr,tr2 in zip(tbody.findAll('tr'),tbody_advanced.findAll('tr')):
        player_stats = [name, position]
        # Basic Stats
        for td in tr.findAll('td'):
            player_stats.append(td.text.encode('utf-8'))
        # Advanced Stats
        for td2 in tr2.findAll('td'):
            player_stats.append(td2.text.encode('utf-8'))
        mywriter.writerow(player_stats)
    file.close()

def main():
    player_names = open('player_names.txt', 'r')
    player_ids = open('player_ids.txt', 'r')
    player_positions = open('player_positions.txt','r')
    
    for name, player_id, position in zip(player_names, player_ids, player_positions):
        print name[:(len(name)-1)] + ', ' + player_id[:(len(player_id)-1)] + ', ' + position[:(len(position)-1)]
        get_stats(name[:(len(name)-1)], player_id[:(len(player_id)-1)], position[:(len(position)-1)])
    
main()

In [5]:
import pandas as pd
import numpy as np

# Columns:
#player,game,date,age,team,location,opponenet,result,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,game,date,age,team,location,opponent,result,GS,MP,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GmSc

df = pd.read_csv('player_stats_1617.csv')
df = df.drop(['Game.1','Age','Age.1','Date.1','Team.1','Location.1','Opponent.1','Result.1','GS.1','MP.1',
             'GmSc.1'], axis=1)

def recode_location(location):
    if location == '@':
        return 1

df.Location = df.Location.apply(recode_location)
df = df[np.isfinite(df['Game'])]
df = df[np.isfinite(df['EFG%'])]
df.Location = df.Location.fillna(0)
df.Location = df.Location.astype('category')

df['3P'] = pd.to_numeric(df['3P'])
df['Fantasy Points'] = df['PTS'] + 0.5 * df['3P'] + 1.2 * df['TRB'] + 1.5 * df['AST'] + 2 * df['STL'] + 2 * df['BLK'] - df['TOV']

# df.to_csv('cleaned_stats.csv')



<class 'pandas.core.frame.DataFrame'>
Int64Index: 27723 entries, 0 to 41895
Data columns (total 43 columns):
Player            27723 non-null object
Position          27723 non-null object
Game              27723 non-null float64
Date              27723 non-null object
Team              27723 non-null object
Location          27723 non-null category
Opponent          27723 non-null object
Result            27723 non-null object
GS                27723 non-null object
MP                27723 non-null object
FG                27723 non-null object
FGA               27723 non-null object
FG%               27723 non-null object
3P                27723 non-null int64
3PA               27723 non-null object
3P%               20512 non-null object
FT                27723 non-null object
FTA               27723 non-null float64
FT%               16139 non-null float64
ORB               27723 non-null float64
DRB               27723 non-null float64
TRB               27723 non-null float64
AST 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('cleaned_stats_1617.csv')
df = df.drop(['Unnamed: 0'], axis=1)

offensive_stats = pd.read_csv('team_offensive_stats1.csv')
defensive_stats = pd.read_csv('team_defensive_stats.csv')


teams_involved = df.loc[:,['Team','Opponent']]
offensive_df = pd.DataFrame(index=range(1), columns=offensive_stats.columns)
defensive_df = pd.DataFrame(index=range(1), columns=defensive_stats.columns)

for game in range(teams_involved.shape[0]):
        offensive_df = offensive_df.append(offensive_stats.loc[offensive_stats['Team'] == teams_involved.iloc[game,0]])
        defensive_df = defensive_df.append(defensive_stats.loc[defensive_stats['Team'] == teams_involved.iloc[game,1]])
        
offensive_df = offensive_df[np.isfinite(offensive_df['Score_off'])]
defensive_df = defensive_df[np.isfinite(defensive_df['Score_def'])]
offensive_df = offensive_df.reset_index()
defensive_df = defensive_df.reset_index()
del offensive_df['index'], defensive_df['index']

team_stats = pd.concat([offensive_df,defensive_df], axis=1)
df = pd.concat([df,team_stats], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27723 entries, 0 to 27722
Data columns (total 65 columns):
Player                    27723 non-null object
Position                  27723 non-null object
Game                      27723 non-null float64
Date                      27723 non-null object
Team                      27723 non-null object
Location                  27723 non-null float64
Opponent                  27723 non-null object
Result                    27723 non-null object
GS                        27723 non-null int64
MP                        27723 non-null object
FG                        27723 non-null int64
FGA                       27723 non-null int64
FG%                       27723 non-null float64
3P                        27723 non-null int64
3PA                       27723 non-null int64
3P%                       20512 non-null float64
FT                        27723 non-null int64
FTA                       27723 non-null float64
FT%                       16

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

df = pd.read_csv('cleaned_stats_1617.csv')

def recode_teams(team):
    if team == 'ATL':
        return 0
    elif team == 'BOS':
        return 1
    elif team == 'BRK':
        return 2
    elif team == 'CHO':
        return 3
    elif team == 'CHI':
        return 4
    elif team == 'CLE':
        return 5
    elif team == 'DAL':
        return 6
    elif team == 'DEN':
        return 7
    elif team == 'DET':
        return 8
    elif team == 'GSW':
        return 9
    elif team == 'HOU':
        return 10
    elif team == 'IND':
        return 11
    elif team == 'LAC':
        return 12
    elif team == 'LAL':
        return 13
    elif team == 'MEM':
        return 14
    elif team == 'MIA':
        return 15
    elif team == 'MIL':
        return 16
    elif team == 'MIN':
        return 17
    elif team == 'NOP':
        return 18
    elif team == 'NYK':
        return 19
    elif team == 'OKC':
        return 20
    elif team == 'ORL':
        return 21
    elif team == 'PHI':
        return 22
    elif team == 'PHO':
        return 23
    elif team == 'POR':
        return 24
    elif team == 'SAC':
        return 25
    elif team == 'SAS':
        return 26
    elif team == 'TOR':
        return 27
    elif team == 'UTA':
        return 28
    elif team == 'WAS':
        return 29
    
def recode_position(position):
    if position == 'PG':
        return 0
    elif position == 'SG':
        return 1
    elif position == 'SF':
        return 2
    elif position == 'PF':
        return 3
    elif position == 'C':
        return 4

def recode_MP(MP):
    if len(MP) <= 5:
        time = datetime.strptime(MP, '%M:%S')
        return time.minute + time.second/60.
    else:
        time = datetime.strptime(MP, '%M:%S:%f')
        return time.minute + time.second/60.

df.Team = df.Team.apply(recode_teams)
df.Opponent = df.Opponent.apply(recode_teams).astype(int)
df.Position = df.Position.apply(recode_position)
df['Minutes'] = df.MP.apply(recode_MP)
df.info()

ValueError: time data '60:07:00' does not match format '%M:%S:%f'